In [1]:
# THE STELLAR INTERPRETATION OF QUANTUM MECHANICS
# THANKS TO ETTORE MAJORANA
#
# Choose:
n=4
# where n is the dimensionality of the randomly 
# generated quantum system, leading to n-1 stars
# in the sky. When n=2, we have a qubit in the familiar
# Bloch sphere representation.

import matplotlib
# NB: You may need to change matplotlib's backend for this to work:
matplotlib.use('WXAgg') 
from matplotlib import cm, colors, animation
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg
import math
import mpmath
import qutip
import sys

# Stereographically projects an XYZ cartesian coordinate on 
# the Riemann sphere to a point on the complex plane.
def flatten(x, y, z):
	if z == 1:
		return Inf 
	else:
		return complex(x/(1-z), y/(1-z))

# Reverse stereographically projects a point on the complex plane
# to an XYZ cartesian coordinate on the Riemann sphere.
def rollup(z):
	x = z.real
	y = z.imag
	return (2*x)/(1+(x**2)+(y**2)),\
		   (2*y)/(1+(x**2)+(y**2)),\
		   (-1+(x**2)+(y**2))/(1+(x**2)+(y**2))

# Takes a complex vector, interprets its ordered components as
# the coefficients of a polynomial in one variable, and solves for
# the roots, giving an unordered set of complex numbers. 
# These complex points are reverse stereographically projected onto 
# the Riemann sphere, and the XYZ cartesian coordinates of these points 
# are returned.
def constellate(v):
	roots = mpmath.polyroots(v)
	xx, yy, zz = [], [], []
	for root in roots:
		x, y, z = rollup(root)
		xx.append(float(x))
		yy.append(float(y))
		zz.append(float(z))
	return xx, yy, zz

# Creates a unit sphere, returning XYZ cartesian coordinates
# which mesh the surface to a given resolution.
def sphere(resolution=30):
	u = np.linspace(0, np.pi, resolution)
	v = np.linspace(0, 2*np.pi, resolution)
	x = np.outer(np.sin(u), np.sin(v))
	y = np.outer(np.sin(u), np.cos(v))
	z = np.outer(np.cos(u), np.ones_like(v))
	return x, y, z

# Returns a randomly generated quantum system:
# a random unit complex vector for the state,
# a random hermitian matrix for the hamiltonian,
# aka the energy operator aka the time evolver.
def bang(n):
	return qutip.rand_ket(n).full().T[0], \
	 	   qutip.rand_herm(n).full().T

# Initialize graphics
fig = plt.figure(dpi=140)
axis = fig.add_subplot(111, projection='3d')

# Initialize quantum simulation
state, hamiltonian = bang(n)

# Plot the celestial sphere, and the starting constellation.
sphere = axis.plot_wireframe(*sphere(), rstride=1, cstride=1, color='b', alpha=0.4)
constellation = axis.scatter3D(*constellate(state), color="r", s=100)

# Rotation animation:
# Continuously increments elevation and azimuth of the camera
# at each time step.
def rotate(angle):
    axis.view_init(elev=angle, azim=angle)
rot_animation = animation.FuncAnimation(fig, rotate, frames=np.arange(0,1000,2), interval=10, repeat=True)

# Time evolution animation:
# Exponentiates the hamiltonian, integrating up to the current
# time step, giving a unitary transformation implementing
# continuous time evolution up to now, which is applied
# to the initial state. The resulting present state is constellated,
# and the locations of the stars in the scatterplot are updated.
def evolve(t):
	global state
	global hamiltonian
	global constellation
	time_step = scipy.linalg.expm(-2*math.pi*complex(0,1)*hamiltonian*t)
	x, y, z = constellate(time_step.dot(state))
	constellation._offsets3d = (x,y,z)
	return constellation,
# Consider adjusting the frames and interval parameters to explore
# time evolution at different scales.
star_animation = animation.FuncAnimation(fig, evolve,\
			frames=np.linspace(0,100,10000), interval=10, repeat=False)

# Restricts the representation of our axes to the unit cube,
# and packs the graphics in our window snugly.
axis.set_xlim([-1,1])
axis.set_ylim([-1,1])
axis.set_zlim([-1,1])
axis.set_aspect("equal")
plt.tight_layout()

# Off to the races!
plt.show()